# Data Clean Up processes

In [1]:
import re, numbers
import numpy as np
import pandas as pd
from functools import partial #currency -> USD?

#Converts Currencies
def currency_conv(curr):
    if   curr == "GBP": return 1.4
    elif curr == "EUR": return 1.24
    elif curr == "USD": return 1.24
    else: return np.NaN

# Mainly used for execution Yr Data
def ckInt(is_int, tf=False, dbg=False, btwnYrs=True):
    if is_int != is_int: return np.NaN
    elif isinstance(is_int, numbers.Number):
        if dbg: print("dbg: numeric")
        if tf: return True
        else:
            #year of execution Not ridiculous
            if btwnYrs and (is_int < 1800 or is_int > 2019): return np.NaN
            return is_int
    else:
        if dbg: print("dbg: Notnum")
        if tf: return False
        else:
            r = re.search("[1-9][0-9][0-9][0-9]", is_int)
            if not(r): return np.NaN
            else:
                nYr = float(r.group(0))
                if btwnYrs and (nYr < 1800 or nYr > 2019): return np.NaN
                return nYr
            
# Using partial functions for quick apply to columnal data
TFInt = partial(ckInt,True, True)
TFint = partial(ckInt,True, False)

In [2]:
# Reading the input Data in
#data = pd.read_csv('data.csv', encoding='iso-8859-1')
data = pd.read_csv('data.csv', encoding='latin-1')
print(data.shape)
data.info()

(107578, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107578 entries, 0 to 107577
Data columns (total 18 columns):
artist_birth_year        107578 non-null int64
artist_death_year        71186 non-null float64
artist_name              107578 non-null object
artist_nationality       107578 non-null object
auction_date             107578 non-null object
category                 107578 non-null object
currency                 107578 non-null object
edition                  13376 non-null object
estimate_high            68404 non-null float64
estimate_low             68409 non-null float64
hammer_price             101254 non-null float64
location                 89733 non-null object
materials                105739 non-null object
measurement_depth_cm     107578 non-null int64
measurement_height_cm    107578 non-null int64
measurement_width_cm     107578 non-null int64
title                    107118 non-null object
year_of_execution        79818 non-null object
dtypes: float64(4)

In [3]:
print(data[['artist_birth_year', 'artist_death_year', 'artist_name','year_of_execution']].head())
#print(data.year_of_execution.value_counts(dropna=False))
#print(data.currency.value_counts(dropna=False))
#print(data.artist_nationality.value_counts(dropna=False))

   artist_birth_year  artist_death_year  artist_name year_of_execution
0               1902             1982.0  Wifredo Lam              1944
1               1902             1982.0  Wifredo Lam              1964
2               1902             1982.0  Wifredo Lam              1955
3               1902             1982.0  Wifredo Lam              1973
4               1902             1982.0  Wifredo Lam               NaN


In [4]:
# Adding currency coversion rates
data['CONVrate'] = data.currency.apply(currency_conv)
# Mark if work is unique == more $$$ than others
data['unique'] = data.edition == "unique"
#auction date post death? or near/at death = death increase price
data['auct_yr'] =  pd.DatetimeIndex(data.auction_date).year
data['exec_pmortem'] = data.auct_yr >= data.artist_death_year
data["yr_exec"] = data.year_of_execution.apply(ckInt)
#print(data.year_of_execution.value_counts(dropna=False)) # check to see if conversion to int done right

In [5]:
print("hammer < highEstm", np.sum(data.estimate_high < data.hammer_price))
print("hammer > low Estm", np.sum(data.estimate_low  > data.hammer_price))
data[['exec_pmortem', 'auct_yr', 'artist_death_year']].query("exec_pmortem == 1").head()

hammer < highEstm 18984
hammer > low Estm 24572


,exec_pmortem,auct_yr,artist_death_year
0,True,1987,1982.0
1,True,2005,1982.0
2,True,1994,1982.0
3,True,1987,1982.0
4,True,2002,1982.0


In [6]:
isinstance(data.year_of_execution.iloc[-1] , numbers.Number)
isinstance(np.NaN, numbers.Number)

True

In [7]:
#birth yr > death yr
print(np.sum(data.artist_birth_year > data.artist_death_year)) # 0 good
#print(data.yr_exec.value_counts())

0


In [8]:
def cnv_mean(x): return float(x.replace(',',''))
#Adding Dji Avg Data Given Acution years goes back to 80's. This will be used as asset value normalizer

dji = pd.read_csv("DJind.txt", sep="\t")
dji['DJlevel'] = dji.ValueClose.apply(cnv_mean)
DJi = dji.groupby(['Year'], as_index=False).agg({'DJlevel': 'mean'})
df = data.merge(DJi, how='left', left_on=['auct_yr'], right_on=['Year'])
del df['Year']
DJi.head()

,Year,DJlevel
0,1900,68.130
1,1901,70.440
2,1902,64.320
3,1903,56.855
4,1904,70.050


In [9]:
def binny(df_series, bins=25):
    bins = np.linspace(df_series.min(), df_series.max(), bins)
    grps = df_series.groupby(np.digitize(df_series, bins))
    return grps.agg(['min', 'max','count'])

## Getting rid of useless Data

In [11]:
df = df[~np.isnan(df.hammer_price)]
df = df[df.hammer_price > 1000]; df.shape  

(85868, 24)

## Adding USD converated & Dow Jones Adjusted Data
### Seriously Data is from 80's & all assets should be normalized

In [12]:
df['USD_hamP'] = df.hammer_price * df.CONVrate
df['USD_Hest'] = df.estimate_high * df.CONVrate
df['USD_Lest'] = df.estimate_low * df.CONVrate
df['USD_DjiP'] = df.USD_hamP / df.DJlevel
df['USD_Dest'] = (df.USD_Hest+df.USD_Lest) / (df.DJlevel*2)
df['USD_estH'] = df.USD_Hest / df.DJlevel

In [15]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=.2)
print(df_train.shape)
print(df_test.shape)

(68694, 30)
(17174, 30)


In [13]:
# Some additional Data checking
"""
DATA Conversion & insight
year of execution -> numeric early, mid, late
attempt to id the artist by yr first 3 digits or by 50 year interval?
Long vs Short title?
check to see if market level (SPX) or volatility? (VIX not that long) has anything to do w/ price?

data.year_of_execution.head().apply(ckInt)
data.year_of_execution.apply(ckInt)
data[['year_of_execution', ]]
data.year_of_execution[data.year_of_execution.apply(TFint)]
data.year_of_execution.apply(TFint)
data.year_of_execution.iloc[-3:].apply(ckInt)
data.year_of_execution.iloc[-3:].apply(TFInt) # what the #$@#$!
data.year_of_execution.iloc[-3:]
ckInt(data.year_of_execution.iloc[-1], True, True)
"""

"\nDATA Conversion & insight\nyear of execution -> numeric early, mid, late\nattempt to id the artist by yr first 3 digits or by 50 year interval?\nLong vs Short title?\ncheck to see if market level (SPX) or volatility? (VIX not that long) has anything to do w/ price?\n\ndata.year_of_execution.head().apply(ckInt)\ndata.year_of_execution.apply(ckInt)\ndata[['year_of_execution', ]]\ndata.year_of_execution[data.year_of_execution.apply(TFint)]\ndata.year_of_execution.apply(TFint)\ndata.year_of_execution.iloc[-3:].apply(ckInt)\ndata.year_of_execution.iloc[-3:].apply(TFInt) # what the #$@#$!\ndata.year_of_execution.iloc[-3:]\nckInt(data.year_of_execution.iloc[-1], True, True)\n"

In [17]:
df_train.columns

Index(['artist_birth_year', 'artist_death_year', 'artist_name',
       'artist_nationality', 'auction_date', 'category', 'currency', 'edition',
       'estimate_high', 'estimate_low', 'hammer_price', 'location',
       'materials', 'measurement_depth_cm', 'measurement_height_cm',
       'measurement_width_cm', 'title', 'year_of_execution', 'CONVrate',
       'unique', 'auct_yr', 'exec_pmortem', 'yr_exec', 'DJlevel', 'USD_hamP',
       'USD_Hest', 'USD_Lest', 'USD_DjiP', 'USD_Dest', 'USD_estH'],
      dtype='object')